### EODHD simple API test

In [1]:
import datetime as dt  # working with dates
import pandas as pd  # working with data frames
import requests  # for "get" request to API

eod_api = "6654fc24c4b833.59735655"

In [2]:
url_get_list_of_tickers = f"https://eodhistoricaldata.com/api/exchange-symbol-list/US?api_token={eod_api}&fmt=json"

# get a DataFrame with symbols on the stock exchange US
df = pd.DataFrame(requests.get(url_get_list_of_tickers).json())

df

,Code,Name,Country,Exchange,Currency,Type,Isin
0,0P0000A412,Franklin U.S. Opportunities Fund A,USA,PINK,USD,FUND,None
1,0P0000RX5G,Franklin Floating Rate Fund PLC W USD Dis,USA,PINK,USD,FUND,None
2,0P0001GXZ7,Oaktree Global Credit Fund P,USA,OTC,USD,FUND,None
3,0P0001NGF5,Liontrust GF Sust Fut MA Glbl B8 Acc USD,USA,PINK,USD,FUND,None
4,A,Agilent Technologies Inc,USA,NYSE,USD,Common Stock,US00846U1016
...,...,...,...,...,...,...,...
47693,ZYME,Zymeworks Inc. Common Stock,USA,NASDAQ,USD,Common Stock,CA98985W1023
47694,ZYXI,Zynex Inc,USA,NASDAQ,USD,Common Stock,US98986M1036
47695,ZZHGF,ZhongAn Online P & C Insurance Co. Ltd,USA,PINK,USD,Common Stock,None
47696,ZZHGY,ZhongAn Online P & C Insurance Co. Ltd,USA,PINK,USD,Common Stock,None


In [3]:
# make a boolean mask to select specific exchange (NYSE, NASDAQ, NYSE ARCA) in US
df = df[df["Exchange"].isin(["NYSE", "NASDAQ", "NYSE ARCA"])]

# get list symbols(numpy array) from DataFrame
list_symbols = df["Code"].values

In [4]:
def request_symbol(symbol):

    req_params = {
        "api_token": eod_api,
        "period": "d",
        "fmt": "json",
        "from": "1950-01-01",
        "to": dt.datetime.now(),
    }

    # send 'get' request with parameters (req_params) and in url substitute (symbol)
    df_stock = pd.DataFrame(
        requests.get(
            f"https://eodhistoricaldata.com/api/eod/{symbol}.US", params=req_params
        ).json()
    )
    return df_stock

In [5]:
def adjusted_values(df_stock):

    df_stock["ratio"] = df_stock["close"] / df_stock["adjusted_close"]

    # recalculate adjusted values open, high, low, close
    df_stock["open"] = df_stock["open"] / df_stock["ratio"]
    df_stock["high"] = df_stock["high"] / df_stock["ratio"]
    df_stock["low"] = df_stock["low"] / df_stock["ratio"]
    df_stock["close"] = df_stock["close"] / df_stock["ratio"]

    return df_stock

In [8]:
symbols = ['TLT','GLD','XLP','XLV','XLK']

for symbol in symbols:
    df_stock = request_symbol(symbol)
    df_stock = adjusted_values(df_stock)

    # load DataFrame in file parquet
    df_stock.to_parquet(f"../data/{symbol}.parquet")